In [117]:
from utils import *
import pandas as pd
from representations import *
# This link: https://www.hockey-reference.com/leagues/ has a list of all seasons played in the NHL

In [120]:
main_page = "https://www.hockey-reference.com/leagues/"
# Get the soup
soup = get_soup(main_page)
# Get all the links in the soup
links = get_all_links(soup)
# Get the links that are seasons
seasons = [link for link in links if "/leagues/NHL_"in link and link.split("_")[-1].split(".")[0].isnumeric()]
# Remove duplicates
seasons = list(set(seasons))
# Sort the seasons
seasons.sort()

web_season = WebSeason("https://www.hockey-reference.com"+seasons[-1],verbose=True)



Found 1311 games


KeyError: 'Visitors'

In [ ]:
# Get the soup for the first season
soup = get_soup(main_page+seasons[0])
a = soup.find('table',id='expanded_standings')


In [ ]:
# Get table from link
# table_link = "https://www.hockey-reference.com/leagues/NHL_1926_standings.html#expanded_standings" # Expanded standings table
table_link = 'https://www.hockey-reference.com/leagues/NHL_2022_games.html#games' # All games of the season
# Using pandas to read the table
df = pd.read_html(table_link)
print(f"Number of tables: {len(df)}")
# Get the first table
df = df[0]
# Get the first 5 rows
# df.head()
print(df)
# Retrieve the first column of the table
col = df.iloc[:,1]
# Extraft the values from the column
col = col.values
# print(len(col))
# print(col)


In [ ]:
# Get the soup of the table link
soup = get_soup(table_link)

In [ ]:
# Find all links under "all_games" table
links = soup.find(id='all_games').find_all('a')

In [ ]:
# Find the hyperlinks to the games of the form "/boxscores/202110010OTT.html"
links = [link.get('href') for link in soup.find(id='all_games').find_all('a') if "/boxscores/" in link.get('href')]
# Remove duplicates
links = list(set(links))
print(links)

In [96]:
#https://www.hockey-reference.com/boxscores/202110120TBL.html
print(links[0])
game_soup = get_soup("https://www.hockey-reference.com"+links[0])
tables = game_soup.find_all('table',id=re.compile(r'(\w+)_skaters'))#|(\w+)_goalies')) # NORMAL TABLES
ADVANCED_KEYS = ["ALLAll","ALL5v5","ALLEV","ALLPP","ALLSH","CLAll","CL5v5"]
advanced_tables = []
for advanced_key in ADVANCED_KEYS:
    advanced_tables.append(game_soup.find_all('table',id=re.compile(r'(\w+)_'+advanced_key)))
# tables = game_soup.find_all('table',id=re.compile(r'(\w+)_adv_ALLAll')) # ADVANCED TABLES
# for table in tables:
#     print(str(table))
ids = re.compile(r'(\w+)_adv_ALLAll')

# for table in tables:
#     print(ids.findall(str(table))[0])
for adv_table in advanced_tables:
    df = pd.read_html(str(adv_table[0]))[0].iloc[-1,:]
    print(df)
    df.to_dict().pop("Player")

# %timeit ids.findall(str(tables[0]))[0]

/boxscores/202110160BUF.html
Player    TOTAL
iCF         NaN
SAT‑F        37
SAT‑A        53
CF%        41.1
CRel%       NaN
ZSO         NaN
ZSD         NaN
oZS%       42.8
HIT          15
BLK          10
Name: 18, dtype: object
Player    TOTAL
iCF         NaN
SAT‑F        27
SAT‑A        44
CF%        38.0
CRel%       NaN
ZSO         NaN
ZSD         NaN
oZS%       36.4
HIT          13
BLK           9
Name: 18, dtype: object
Player    TOTAL
iCF         NaN
SAT‑F        31
SAT‑A        47
CF%        39.7
CRel%       NaN
ZSO         NaN
ZSD         NaN
oZS%       40.0
HIT          13
BLK           9
Name: 18, dtype: object
Player    TOTAL
iCF         NaN
SAT‑F         6
SAT‑A         1
CF%        85.7
CRel%       NaN
ZSO         NaN
ZSD         NaN
oZS%      100.0
HIT           1
BLK           0
Name: 18, dtype: object
Player    TOTAL
iCF         NaN
SAT‑F         0
SAT‑A         5
CF%         0.0
CRel%       NaN
ZSO         NaN
ZSD         NaN
oZS%        0.0
HIT           1
BLK        

In [92]:
print(len(advanced_tables[0]))

2


In [98]:
# Read the tables with pandas
tables = [pd.read_html(str(table))[0] for table in tables]
print(len(tables))

2


In [115]:
print(tables[1].iloc[-1,:])
key = "TOT"
d = tables[1].iloc[-1,:]
# Remove Assists
d.pop("Assists")
d.pop("Scoring")
d = d.to_dict()
print(d)
# Replace keys with key_name
d = {key+"_"+k if not isinstance(k,tuple) else key+"_"+k[-1]:v if v!=v else v for k,v in d.items()}
# d = {k:0 if v!=v else v for k,v in d.items()}
print(d)
d = {k:v/100 if k.endswith("%") else v for k,v in d.items()}
print(d)
d.pop(key+'_Player')
print(d)
# Replace all nan with 0
# print(tables[1].iloc[-1,:]["Unnamed: 14_level_0"]["S"])
# print(tables[0].iloc[-1,:]["Unnamed: 14_level_0"]["S"])


Unnamed: 0_level_0   Rk          NaN
Unnamed: 1_level_0   Player    TOTAL
Scoring              G             1
                     A             2
                     PTS           3
Unnamed: 5_level_0   +/-         NaN
Unnamed: 6_level_0   PIM           4
Goals                EV            0
                     PP            1
                     SH            0
                     GW          NaN
Assists              EV          NaN
                     PP          NaN
                     SH          NaN
Unnamed: 14_level_0  S            33
Unnamed: 15_level_0  S%          3.0
Unnamed: 16_level_0  SHFT        NaN
Unnamed: 17_level_0  TOI         NaN
Name: 19, dtype: object
{('Unnamed: 0_level_0', 'Rk'): nan, ('Unnamed: 1_level_0', 'Player'): 'TOTAL', ('Unnamed: 5_level_0', '+/-'): nan, ('Unnamed: 6_level_0', 'PIM'): 4, ('Goals', 'EV'): 0, ('Goals', 'PP'): 1, ('Goals', 'SH'): 0, ('Goals', 'GW'): nan, ('Unnamed: 14_level_0', 'S'): 33, ('Unnamed: 15_level_0', 'S%'): 3.0, ('Unnamed